# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.124089e+02     1.501167e+00
 * time: 0.6385419368743896
     1     1.010639e+01     9.031554e-01
 * time: 1.724782943725586
     2    -1.263490e+01     9.840851e-01
 * time: 2.307387113571167
     3    -3.473784e+01     7.193209e-01
 * time: 3.107499122619629
     4    -4.820621e+01     4.879385e-01
 * time: 3.9196510314941406
     5    -5.734739e+01     1.937919e-01
 * time: 4.872514009475708
     6    -6.002847e+01     1.219501e-01
 * time: 5.421026945114136
     7    -6.098501e+01     5.799827e-02
 * time: 5.97014594078064
     8    -6.134788e+01     7.144321e-02
 * time: 6.519359111785889
     9    -6.162246e+01     4.487731e-02
 * time: 7.073666095733643
    10    -6.181112e+01     2.726836e-02
 * time: 8.004288911819458
    11    -6.195821e+01     2.154707e-02
 * time: 8.54223108291626
    12    -6.202360e+01     1.671919e-02
 * time: 9.080785036087036
    13    -6.209379e+01     1.234975e-02
 * time: 9.625065088272095
    14 

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671075
    AtomicLocal         -18.8557694
    AtomicNonlocal      14.8522649
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485379 
    Xc                  -19.3336823

    total               -62.261666452026
